In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
import s3fs

import os
import json

In [ ]:
sagemaker_session = sagemaker.Session()

bucket = 'animal-reidentification'

role = sagemaker.get_execution_role()

In [ ]:
print(bucket, role)
print(os.getcwd())

In [ ]:
data_location = f's3://{bucket}/data/'
annotations_location = f's3://{bucket}/annotations/'

print(f'data_location: {data_location}')
print(f'annotations_location: {annotations_location}')

## Kick off the instance

In [ ]:
# Set hyperparameters
model_name = 'seg_noAug'
batch_size = 64
num_epochs = 15

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='denseNet201_v5_augs.py',
                    source_dir='./kevin/',
                    framework_version='1.8.0',
                    role=role,
                    py_version='py3',
                    instance_count=1,
                    instance_type='ml.g4dn.xlarge', # Sagemaker-managed instance
                    hyperparameters={
                        'batch-size': batch_size,
                        'use-seg': True,
                        'epochs': num_epochs,
                        'name': f'{model_name}_{num_epochs}epoch_batch{batch_size}',
                    })

estimator.fit({
    'data': data_location,
    'annotations': annotations_location
})